## Importing required libraries

In [1]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Import dataset

In [15]:

dataset = pd.read_csv("transformed_data.csv")

In [16]:
dataset.head()

,Unnamed: 0,name,MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,MDVP:Shimmer(dB),Shimmer:APQ3,Shimmer:APQ5,MDVP:APQ,Shimmer:DDA,status,RPDE,spread1,spread2,D2,PPE
0,0,phon_R01_S01_1,-4.848516,-9.567015,-5.599422,-5.195761,-4.501711,-3.129492,-0.853316,-3.824928,-3.464137,-3.516272,-2.726469,1,0.414783,-4.813031,0.266482,2.301442,0.284654
1,1,phon_R01_S01_2,-4.637693,-9.433484,-5.370888,-4.967576,-4.272993,-2.791323,-0.468405,-3.462860,-3.097101,-3.130865,-2.364141,1,0.458359,-4.075192,0.335590,2.486855,0.368674
2,2,phon_R01_S01_3,-4.556380,-9.315701,-5.213976,-4.852350,-4.114751,-2.950185,-0.729811,-3.591027,-3.255021,-3.327018,-2.492536,1,0.429895,-4.443179,0.311173,2.342259,0.332634
3,3,phon_R01_S01_4,-4.608175,-9.315701,-5.294325,-4.964706,-4.196377,-2.901878,-0.659712,-3.532218,-3.217627,-3.277565,-2.433719,1,0.434969,-4.117501,0.334147,2.405554,0.368975
4,4,phon_R01_S01_5,-4.355190,-9.115030,-5.028290,-4.701681,-3.929169,-2.744974,-0.537854,-3.355268,-3.031359,-3.108901,-2.256656,1,0.417356,-3.747787,0.234513,2.332180,0.410335


In [17]:
dataset.columns

Index(['Unnamed: 0', 'name', 'MDVP:Jitter(%)', 'MDVP:Jitter(Abs)', 'MDVP:RAP',
       'MDVP:PPQ', 'Jitter:DDP', 'MDVP:Shimmer', 'MDVP:Shimmer(dB)',
       'Shimmer:APQ3', 'Shimmer:APQ5', 'MDVP:APQ', 'Shimmer:DDA', 'status',
       'RPDE', 'spread1', 'spread2', 'D2', 'PPE'],
      dtype='object')

In [18]:
dataset = dataset.drop(columns=['Unnamed: 0', 'name'], axis = 1)

In [19]:
x = dataset.drop(["status"], axis = 1)

In [20]:
y = dataset["status"]

In [21]:
x.shape

(195, 16)

In [22]:
y.shape

(195,)

## Splitting the dataset into the Training set and Test set

In [25]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 1)

## Training XGBoost on the Training set

In [30]:
from xgboost import XGBClassifier
classifier = XGBClassifier()
classifier.fit(x_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

## Making the Confusion Matrix

In [31]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[ 6  4]
 [ 2 27]]


0.8461538461538461

## Applying k-Fold Cross Validation

In [32]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = x_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Accuracy: 89.71 %
Standard Deviation: 6.54 %
